# Course Overview:
1. Algorithm Trading Basic
2. API Basics and Course Configuration

## Projects:

1. Equal-weight S&P 500 Screener
2. Quantitative Momentum Screener
3. Quantitative Value Screener

### Introduction
Algorithm Trading means using computers to make investment decisions.

Python is the most popular language used in quantitative trading. However, it is slow and sometimes is used to trigger functionality that actually runs in other faster programming languages.

The process of running a quantitative investing strategy can be broken down into the following steps:
1. Collect data
2. Develop a hypothesis for a strategy
3. Backtest that strategy
4. Implement the strategy in production

Because this is an introductory couse, it will differ from production algorithmic trading in 3 major ways:
1. We will be using random data
2. We will not be executing trades
3. We will be saving recommended trades into Excel files

### What is an API?
An API is an Application Programming Interface. APIs allow you to interact with someone else's software using your own code

In this couse, we will be using the IEX Cloud API to gather stock market data to make investment decisions.
API Functionality:
1. GET: get data from API
2. POST: add data to the database exposed by the API
3. PUT: add and overwrite data in the database exposed by the API
4. DELETE: delete data from the API's database


In [ ]:
symbol = 'AAPL'
api_url = f''
data = requests.get(api_url).json()

## Project 1: Equal-weight S&P 500 Screener

S&P 500 is the world's most popular stock market index. Many investment funds are benchmarked to the S&P 500 (top 500 stocks in the US). This means that they seek to replicate the performance of this index by owning all stocks that are held in the index.

One of the most important characteristics of the S&P 500 is that it is market capitalization-weighted. This means that larger companies get a correspondingly larger weight in the index.

In the first project, you will build an alternative version of the S&P 500 where each company has the same weighting.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constitute you should purchase to get an equal-width version of the index fund.

In [4]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [31]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks['Ticker']

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 505, dtype: object

Acquiring an API Token & GET requests from API

In [33]:
def get_price_and_market_cap(symbol):
    IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
    api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"

    data = requests.get(api_url).json()

    # get price and market of a particular stock
    price = data['latestPrice']
    market_cap = data['marketCap']
    return price, market_cap

In [40]:
# adding stocks data to pd 
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker']:
    price, market_cap = get_price_and_market_cap(stock)
    final_df = final_df.append(pd.Series([stock, price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

In [42]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,153.62,45766682272,N/A
1,AAL,21.10,13827317945,N/A
2,AAP,213.86,14396615171,N/A
3,AAPL,151.30,2490497133113,N/A
4,ABBV,121.60,215293874673,N/A
...,...,...,...,...
500,YUM,122.00,36038373425,N/A
501,ZBH,162.09,34221265674,N/A
502,ZBRA,574.33,29413310254,N/A
503,ZION,53.25,8923541749,N/A


Using Batch API Calls to Improve Performance

In [43]:
# Dividing a list into chunks of n objects
def chunks(lst, n):
    '''Yield successive n-sized chunks from lst'''
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [48]:
stock_groups = list(chunks(stocks['Ticker'], 100))
stock_strings = []
for i in range(0, len(stock_groups)):
    stock_strings.append(','.join(stock_groups[i]))

final_df = pd.DataFrame(columns = my_columns)
for stock_string in stock_strings:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_url).json()
    for stock in stock_string.split(','):
        final_df = final_df.append(pd.Series([stock, data[stock]['quote']['latestPrice'], data[stock]['quote']['marketCap'], 'N/A'], index=my_columns), ignore_index=True)

In [49]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,154.60,46098759446,N/A
1,AAL,20.91,13753698964,N/A
2,AAP,214.58,14048090291,N/A
3,AAPL,146.85,2491245714403,N/A
4,ABBV,120.61,208845947458,N/A
...,...,...,...,...
500,YUM,121.20,35834351363,N/A
501,ZBH,166.82,33454068570,N/A
502,ZBRA,574.32,29766113472,N/A
503,ZION,52.75,8979224373,N/A


## Project 2: Momentum Investing Project 

Imagine that you have the choice between investing in two stocks that have had the following returns over the last year:
- AAPL: 35%
- MSFT: 20%

A momentum onvesting strategy would suggest investing in Apple because of its higher recent price return. There are many other nuances to momentum investing strategies that we will explore later.

## Project 3: Quantitative Value Screener

Value investing means investing in stocks that are trading below their perceived instrisic value.
Creating algorithmic value investing strategies relies on a concept called mulitples.

Multiples are calculated by dividing a company's stock price by some measure of the company's worth-like earnings or assets.

Here are a few examples of common multiples used in value investing:
- Price-to-earnings
- Price-to-book-value
- Price-to-free-cash-flow

Each of the individual multiples used by value investors has its pros and cons. One way to minimize the impact of any specific multiple is by using a composite.

We will use a composite of 5 different value metrics in our strategy.